In [ ]:
# https://dssg.github.io/hitchhikers-guide/curriculum/software/python_sql/

import os
from sqlalchemy import create_engine
import pandas as pd
import functools as ft
from matplotlib import pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

### Set up the db connection

In [ ]:
# Get credentials from environment variables
user = os.getenv("PGUSER")
password = os.getenv("PGPASSWORD")
host = os.getenv("PGHOST")
port = os.getenv("PGPORT")
database = "vibrant-routing"

# Configure connection to postgres
engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database)
)

# Open a connect
db_conn = engine.connect()

### Peek into the table

In [ ]:
# Count the number of rows
sql = """
    select
    count(*) as num_rows
    from raw.vibrant_centers_calls_202206031630 vcc
"""
num_rows = pd.read_sql(sql, db_conn)

In [ ]:
# Count the number of columns
sql = """
    select
    count(*) as num_cols
    from information_schema.columns
    where table_name = 'vibrant_centers_calls_202206031630';
"""
num_cols = pd.read_sql(sql, db_conn)


shape = pd.concat([num_rows, num_cols], axis=1, join="inner")
shape

In [ ]:
# Peeks at the first few rows

sql = """
    select * from raw.vibrant_centers_calls_202206031630 vcc
    limit 5
"""

data = pd.read_sql(sql, db_conn)
data.head()

In [ ]:
data.columns

### Sanity checks -- TODO
* completed_at_center = abandoned_at_center + answered_at_center
* (call time == 0) == (answered_at_center == 0)
* ...

### Identify states with local, spanish, and national call centers
The count is 53, consisting of:
* all fifty states
* DC -- Washington, DC (District of Columbia) is the nation's capital city and is not currently a state of its own.
* PR -- Puerto Rico is the most populous of the US Territories, which are lands scattered throughout the Caribbean Sea and Pacific Ocean that the US has jurisdiction over but are not states of their own.
* GU -- Guam is another one of the US Territories.

In [ ]:
# Identify the types of call centers

sql = """
    select
    distinct network
    from raw.vibrant_centers_calls_202206031630 vcc
"""

result = pd.read_sql(sql, db_conn)
result

### Count the number of states with each type of call center

In [ ]:
# Number of states with any call centers (either local/national)
sql = """
    select
        count(distinct center_state) as num_any
    from raw.vibrant_centers_calls_202206031630 vcc
"""
count_of_states_all = pd.read_sql(sql, db_conn)

In [ ]:
# Number of states with local english call centers
sql = """
    select
        count(distinct center_state) as num_eng_local
    from raw.vibrant_centers_calls_202206031630 vcc
    where network = 'NSPL'
"""
count_of_states_english = pd.read_sql(sql, db_conn)

In [ ]:
# Number of states with local spanish call centers
sql = """
    select
        count(distinct center_state) as num_spa_local
    from raw.vibrant_centers_calls_202206031630 vcc
    where network = 'Spanish'
"""
count_of_states_spanish = pd.read_sql(sql, db_conn)

In [ ]:
# Number of states with national call centers
sql = """
    select
        count(distinct center_state) as num_national
    from raw.vibrant_centers_calls_202206031630 vcc
    where network = 'National-Backup'
"""
count_of_states_national = pd.read_sql(sql, db_conn)

In [ ]:
# Compile the above queries into one table

dfs = [
    count_of_states_all,
    count_of_states_english,
    count_of_states_spanish,
    count_of_states_national,
]

result = pd.concat(dfs, axis=1, join="inner")
result

In [ ]:
# List of the states with spanish call centers

sql = """
    select
        distinct center_state as list_of_distinct_states
    from raw.vibrant_centers_calls_202206031630 vcc
    where network = 'Spanish'
    order by center_state
"""

list_of_states_spanish = pd.read_sql(sql, db_conn)
list_of_states_spanish.to_numpy().flatten()

In [ ]:
# List of the states with national call centers

sql = """
    select
        distinct center_state as list_of_distinct_states
    from raw.vibrant_centers_calls_202206031630 vcc
    where network = 'National-Backup'
    order by center_state
"""

list_of_states_national = pd.read_sql(sql, db_conn)
list_of_states_national.to_numpy().flatten()

### Map the call centers

In [ ]:
# Query the center, caller, latlong
sql = "select center_key, caller_state, center_lat, center_lng  \
        from raw.vibrant_centers_calls_202206031630 vcc \
        group by 1, 2, 3, 4; "

df = db_conn.execute(sql)
df = pd.DataFrame(df)
df.head()

In [ ]:
# Generate the map

geometry = [Point(xy) for xy in zip(df["center_lng"], df["center_lat"])]
gdf = GeoDataFrame(df, geometry=geometry)

# This is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
north_america = world[world.continent == "North America"]
gdf.plot(
    ax=north_america.plot(figsize=(40, 30)), marker="o", color="red", markersize=15
);

### Count the number of each type of call center in each state

In [ ]:
# Number of local english centers
sql = """
	select
		center_state,
		count(distinct center_key) as count_eng_local_centers
	from raw.vibrant_centers_calls_202206031630 vcc
	where network = 'NSPL'
	group by center_state
	order by count_eng_local_centers desc
"""
count_eng_local_centers = pd.read_sql(sql, db_conn)

In [ ]:
# Number of local spanish centers
sql = """
	select
		center_state,
		count(distinct center_key) as count_span_local_centers
	from raw.vibrant_centers_calls_202206031630 vcc
	where network = 'Spanish'
	group by center_state
	order by count_span_local_centers desc
"""
count_span_local_centers = pd.read_sql(sql, db_conn)

In [ ]:
# Number of national backup centers
sql = """
	select
		center_state,
		count(distinct center_key) as count_national_centers
	from raw.vibrant_centers_calls_202206031630 vcc
	where network = 'National-Backup'
	group by center_state
	order by count_national_centers desc
"""
count_national_centers = pd.read_sql(sql, db_conn)

In [ ]:
# Compile the above queries into one table
dfs = [count_eng_local_centers, count_span_local_centers, count_national_centers]

count_all = ft.reduce(
    lambda left, right: pd.merge(left, right, how="outer", on="center_state"), dfs
).fillna(0)
count_all.head(10)

### Count the in-state and out-state calls being made, answered, and missed in each state

In [ ]:
# Number of calls being initiated from each state
sql = """
	select
		caller_state,
		count(caller_state) as calls_from_state
	from raw.vibrant_centers_calls_202206031630 vcc
	group by caller_state
	order by calls_from_state desc
"""
calls_from_state = pd.read_sql(sql, db_conn)

In [ ]:
# Number of calls answered by in-state english local centers
sql = """
    select
        center_state,
        count(center_state) as in_state_calls_eng_answered
    from raw.vibrant_centers_calls_202206031630 vcc
    where vcc.answered_at_center = 1 and vcc.center_state = vcc.caller_state and vcc.network = 'NSPL'
    group by center_state
    order by in_state_calls_eng_answered desc
"""
in_state_calls_eng_answered = pd.read_sql(sql, db_conn)

In [ ]:
# Number of calls answered by in-state spanish local centers
sql = """
    select
        center_state,
        count(center_state) as in_state_calls_spa_answered
    from raw.vibrant_centers_calls_202206031630 vcc
    where vcc.answered_at_center = 1 and vcc.center_state = vcc.caller_state and vcc.network = 'Spanish'
    group by center_state
"""
in_state_calls_spa_answered = pd.read_sql(sql, db_conn)

In [ ]:
# Number of in-state call attempts missed (including re-routes)
# (answered by another state's local centers or by National Backup)

sql = """
    select
        center_state,
        count(center_state) as num_in_state_missed
    from raw.vibrant_centers_calls_202206031630 vcc
    where vcc.answered_at_center = 0 and vcc.center_state = vcc.caller_state or vcc.network = 'National-Backup'
    group by center_state
"""
in_state_calls_missed = pd.read_sql(sql, db_conn)

In [ ]:
# Number of out-of-state calls that each state's local centers absorbed
sql = """
    select
        center_state,
        count(center_state) as num_out_state_answers
    from raw.vibrant_centers_calls_202206031630 vcc
    where vcc.answered_at_center = 1 and vcc.center_state <> vcc.caller_state and vcc.network <> 'National-Backup'
    group by center_state
"""
out_state_calls_answered = pd.read_sql(sql, db_conn)

In [ ]:
# Compile the above queries into one table

dfs = [
    in_state_calls_eng_answered,
    in_state_calls_spa_answered,
    in_state_calls_missed,
    out_state_calls_answered,
]
count_all = ft.reduce(
    lambda left, right: pd.merge(left, right, how="outer", on="center_state"), dfs
).fillna(0)

count_all.merge(
    calls_from_state, how="outer", left_on="center_state", right_on="caller_state"
)

count_all.head(10)

### Talk time analysis

Excludes 0-second convos (which indicate that the call was not answered there)

In [ ]:
# Mean and median talk time in local english centers
sql = """
    select
        center_state,
        avg(talk_time_center)/60 as mean_talk_local_eng_mins,
        percentile_cont(0.5) within group(order by talk_time_center/60) as median_talk_local_eng_mins
    from raw.vibrant_centers_calls_202206031630 vcc
    where talk_time_center > 0 and network = 'NSPL'
    group by center_state
"""

talk_time_local_eng_mins = pd.read_sql(sql, db_conn)

In [ ]:
# Mean and median talk time in local spanish centers
sql = """
    select
        center_state,
        avg(talk_time_center)/60 as mean_talk_local_spa_mins,
        percentile_cont(0.5) within group(order by talk_time_center/60) as median_talk_local_spa_mins
    from raw.vibrant_centers_calls_202206031630 vcc
    where talk_time_center > 0 and network = 'Spanish'
    group by center_state
"""

talk_time_local_spa_mins = pd.read_sql(sql, db_conn)

In [ ]:
# Mean and median talk time in national backup centers
sql = """
    select
        center_state,
        avg(talk_time_center)/60 as mean_talk_national_mins,
        percentile_cont(0.5) within group(order by talk_time_center/60) as median_talk_national_mins
    from raw.vibrant_centers_calls_202206031630 vcc
    where talk_time_center > 0 and network = 'National-Backup'
    group by center_state
"""

talk_time_national_mins = pd.read_sql(sql, db_conn)

In [ ]:
# Compile the above queries into one table

dfs = [talk_time_local_eng_mins, talk_time_local_spa_mins, talk_time_national_mins]
talk_times = ft.reduce(
    lambda left, right: pd.merge(left, right, how="outer", on="center_state"), dfs
).fillna(0)

talk_times.head(10)

### Total talk/ring/wait times (sum)

Note that in theory, it should be the case that ring_time_center == time_to_answer_center when the call is answered at the center. However, we can see that this is not the case in practice. This is because both these metrics are not directly recorded, but rather calculated via proxies. They shouldn't be too terribly different.

In [ ]:
sql = """
    select
        call_key ,
        caller_state ,
        sum(talk_time_center) as sum_talk_time ,
        sum(ring_time_center) as sum_ring_time ,
        sum(time_to_answer_center) as sum_time_to_answer_center ,
        sum(time_to_leave_center) as sum_time_to_leave_center ,
        sum(time_to_abandon_center) sum_time_to_abandon ,
        max_attempt_num,
        max(attempt_number) as max_attempts_sanity_check ,
        max(abandoned_at_center) as ever_abandoned ,
        max(dc_at_center) as ever_dc ,
        max(bs_at_center) as ever_bs ,
        max(na_at_center) as ever_na ,
        sum(answered_in_state) as answered_in_state ,
        sum(answered_out_state) as answered_out_state ,
        sum(in_state_exception) as in_state_exception 
    from raw.vibrant_centers_calls_202206031630 vcc 
    where talk_time_center > 0
    group by
        call_key ,
        caller_state ,
        center_key ,
        max_attempt_num 
"""

total_times = pd.read_sql(sql, db_conn)

In [ ]:
total_times.sort_values("sum_ring_time", ascending=False, inplace=True)
total_times

### Time to abandon

In [ ]:
# Calls abandoned before being picked up by the first call center (mins)
sql = """
    select
        time_to_abandon_center/60 as time_to_abandon_at_first
    from raw.vibrant_centers_calls_202206031630 vcc
    where network <> 'National-Backup' and attempt_number = 1 and time_to_abandon_center > 0
"""

time_to_abandon_at_first = pd.read_sql(sql, db_conn).to_numpy().flatten()

In [ ]:
# Calls abandoned before being picked up by any call center (mins) -- not a sum

sql = """
    select
        time_to_abandon_center/60 as time_to_abandon_all
    from raw.vibrant_centers_calls_202206031630 vcc
    where time_to_abandon_center > 0 and time_to_abandon_center < 200*60
"""

time_to_abandon_all = pd.read_sql(sql, db_conn).to_numpy().flatten()

In [ ]:
plt.boxplot([time_to_abandon_at_first, time_to_abandon_all])
plt.xticks(
    [1, 2], ["All data\nabandoned at\nfirst center", "All data\nabandoned\nanywhere"]
)
plt.ylabel("Time to abandon (mins)")
plt.title(
    "Abandoned calls: Time before caller hung up without speaking to anyone\nData ranges from Jan 2019-Present"
)

caption = 'Note: the boxplot for "All data" was truncated in order to fit on this chart\ns.t. calls where time_to_answer_center >= 200mins are not pictured (a handful of datapoints were excluded).'
plt.figtext(0.5, -0.1, caption, wrap=True, horizontalalignment="center", fontsize=12)

plt.show()

### Secondary data -- state metrics

In [ ]:
# Query some basic state stats

sql = """
    select
        state_abbrev,
        population_2019,
        suicides_2020
    from raw.state_data sd 
"""


state_metrics = pd.read_sql(sql, db_conn)
state_metrics.head()

### EDA Questions for Technical Formulation (07/05)

#### Number of Routing Attempts per Month in 2020 & 2021

In [ ]:
sql = """
    select extract(month from ra.arrived_datetime_est::date) as "month", count(*) as "routing_attempts_per_month"
    from processed.routing_attempts ra
    where (extract(year from ra.arrived_datetime_est::date) = 2020) or (extract(year from ra.arrived_datetime_est::date) = 2021)
    group by 1
    order by 1 asc;
"""

num_routing_attemps_by_month = pd.read_sql(sql, db_conn)
num_routing_attemps_by_month

#### Number of Distinct Calls per Month

In [ ]:
sql = """
     select extract(month from ra.arrived_datetime_est::date) as "month", count(distinct call_key) as "num_distinct_calls_per_month"
    from processed.routing_attempts ra
    where (extract(year from ra.arrived_datetime_est::date) = 2020) or (extract(year from ra.arrived_datetime_est::date) = 2021)
    group by 1
    order by 1 asc;
"""

num_distinct_calls_by_month = pd.read_sql(sql, db_conn)
num_distinct_calls_by_month

#### Overall Call Answer Rate by Month

In [ ]:
sql = """
    select extract(month from ra.arrived_datetime_est::date) as "month", sum(ra.answered_at_center)::float/count(distinct ra.call_key)::float as answer_rate 
    from processed.routing_attempts ra
    group by 1
    order by 1 asc;
"""
answer_rate_by_month = pd.read_sql(sql, db_conn)
answer_rate_by_month

#### Answer Rate on First Attempt by Month

In [ ]:
sql = """
    with table_1 as
        (select extract(month from arrived_datetime_est::date) as "month",   sum(aa.answered_at_center)::float as answer_at_first_attempt
        from processed.routing_attempts aa
        where aa.attempt_number =1
        group by 1
        order by 1 asc) , 
	table_2 as 
        (select extract(month from arrived_datetime_est::date) as "month",   count(distinct aa.call_key)::float as distinct_call_keys
        from processed.routing_attempts aa
        group by 1
        order by 1 asc)
	select "month",  answer_at_first_attempt/distinct_call_keys as answer_rate_on_first_attempt
	from table_1 join table_2 using ("month")
	order by "month" asc;
"""

answer_rate_on_first_attempt = pd.read_sql(sql, db_conn)
answer_rate_on_first_attempt

#### Histogram of number of routing attemps

In [ ]:
sql = """
    select count(distinct call_key), max_attempt_num
    from processed.routing_attempts ra 
    group by 2
    order by 2 asc;
"""

hist_num_routing_attempts = pd.read_sql(sql, db_conn)
hist_num_routing_attempts

#### Answer and Abandonment Rates by Routing Attempt

In [ ]:
sql = """
    select 
	answered_at_center::float / count_call_keys::float as answer_rate,
	abandoned_at_center::float / count_call_keys::float as abandon_rate,
	max_attempt_num,
	answered_at_center,
	abandoned_at_center,
	count_call_keys
from
	(select
		max_attempt_num,
		sum(answered_at_center) as answered_at_center,
		sum(abandoned_at_center) as abandoned_at_center,
		count(distinct call_key) as count_call_keys
	from processed.routing_attempts ra
	group by
		max_attempt_num
	) as answer_and_abandon_counts_by_attempt_num 
;
"""

answer_and_abandon_rates_by_routing_attempt = pd.read_sql(sql, db_conn)
answer_and_abandon_rates_by_routing_attempt

## Waiting times

In [ ]:
sql = """
    select center_key , termination_number ,
        center_is_acd , center_is_aa , count(distinct call_key) as num_calls,
        max(extract(year from arrived_datetime_est)) ,
        min(time_to_leave_center) / 60 as min_time_to_leave_center, 
        avg(time_to_leave_center) / 60 as avg_time_to_leave_center, 
        percentile_disc(0.5) within group (order by time_to_leave_center) / 60  as median,
        max(time_to_leave_center) / 60 as max_time_to_leave_center, 
        stddev(time_to_leave_center) / 60 as std_time_to_leave_center
    from processed.routing_attempts ra  
    where flowout_from_center = 1 and answered_at_center = 0 and completed_at_center = 0 and abandoned_at_center = 0
    group by center_key , termination_number , center_is_acd , center_is_aa
    order by max(time_to_leave_center) desc, center_key desc;
"""

center_wait_time = pd.read_sql(sql, db_conn)

In [ ]:
center_wait_time_ACD = center_wait_time[center_wait_time["center_is_acd"] == 1]
center_wait_time_ACD_and_AA = center_wait_time_ACD[
    center_wait_time_ACD["center_is_aa"] == 1
]
center_wait_time_nonACD = center_wait_time[center_wait_time["center_is_acd"] == 0]
center_wait_time_nonACD_nonAA = center_wait_time_nonACD[
    center_wait_time_nonACD["center_is_aa"] == 0
]

center_timings = pd.concat(
    [
        center_wait_time_ACD["median"].describe(),
        center_wait_time_ACD_and_AA["median"].describe(),
        center_wait_time_nonACD["median"].describe(),
        center_wait_time_nonACD_nonAA["median"].describe(),
    ],
    axis=1,
)
center_timings.set_axis(
    ["ACD", "ACD & AA", "non-ACD", "non-ACD & non-AA"], axis="columns", inplace=True
)

In [ ]:
center_timings

We ask Rebecca to shed light on the outliers. Our understanding was that ACD centers are capable of holding calls for a max of 3 minutes while non-ACD centers can hold them for a max of 1 minute before flowing them to the next center.